In [1]:
from langchain_core.prompts import ChatPromptTemplate

from dotenv import load_dotenv

load_dotenv()

True

In [2]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")

In [3]:
prompt_val = prompt.invoke({"topic": "dog"})
print(prompt_val)

messages=[HumanMessage(content='Tell me an interesting fact about dog', additional_kwargs={}, response_metadata={})]


In [4]:
print(prompt_val.to_messages())

[HumanMessage(content='Tell me an interesting fact about dog', additional_kwargs={}, response_metadata={})]


In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")
result = model.invoke(prompt_val)
result

AIMessage(content='An interesting fact about dogs is that they have an extraordinary sense of smell—estimated to be between 10,000 to 100,000 times more acute than that of humans. This incredible olfactory ability allows dogs to detect various scents, which is why they are often used in search and rescue operations, as detection dogs for drugs or explosives, and even in medical settings to identify certain diseases in humans by sniffing breath or skin. Their sense of smell is so powerful that they can even identify specific emotions in people based on their scent!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 14, 'total_tokens': 122, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'i

In [6]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(result)

'An interesting fact about dogs is that they have an extraordinary sense of smell—estimated to be between 10,000 to 100,000 times more acute than that of humans. This incredible olfactory ability allows dogs to detect various scents, which is why they are often used in search and rescue operations, as detection dogs for drugs or explosives, and even in medical settings to identify certain diseases in humans by sniffing breath or skin. Their sense of smell is so powerful that they can even identify specific emotions in people based on their scent!'

### Now let´s do this LCEL

In [7]:
prompt = ChatPromptTemplate.from_template("Tell me an interesting fact about {topic}")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

basicchain = model | output_parser

In [8]:
basicchain.invoke("hello!")

'Hello! How can I assist you today?'

In [9]:
chain = prompt | model | output_parser

chain.invoke({"topic": "dog"})

'One interesting fact about dogs is that they have an extraordinary sense of smell, which is estimated to be between 10,000 and 100,000 times more sensitive than that of humans. This incredible olfactory ability allows them to detect certain diseases, locate lost persons, and even discern emotions through scent. This is why dogs are often used in search and rescue missions, as well as in medical detection roles, such as sniffing out cancer or diabetic episodes.'

### Most important methods of the Runnable Interface (Stream, Invoke, Batch)

In [10]:
for s in chain.stream({"topic": "bears"}):
    print(s, end="", flush=True)

One interesting fact about bears is that they can be classified into two main subfamilies: Ursinae (the true bears) and Ailuropodinae (which includes the giant panda). While true bears, like grizzly and polar bears, are predominantly carnivorous or omnivorous, giant pandas have a diet that consists almost entirely of bamboo—despite being classified as a bear. This dietary specialization is so pronounced that giant pandas have adapted unique digestive systems and molar teeth suited for crushing and processing tough bamboo, making them an intriguing exception within the bear family!

In [ ]:
chain.invoke({"topic": "bears"})

In [12]:
chain.batch([{"topic": "bears"}, {"topic": "cats"}])

['One interesting fact about bears is that they have an exceptional sense of smell, which is estimated to be seven times better than that of a bloodhound. This incredible olfactory ability allows them to detect food sources from miles away and is crucial for their survival, especially during the foraging season. For example, polar bears can smell seals beneath several feet of compacted snow, helping them locate their prey even in harsh Arctic conditions.',
 'One interesting fact about cats is that they have a unique grooming behavior known as "allogrooming," where they groom each other to strengthen social bonds. This behavior is not just about cleanliness; it also helps to reduce stress and establish hierarchies within cat communities. Additionally, when cats groom themselves, they spread natural oils in their fur that help keep it healthy and insulated.']

In [13]:
async for s in chain.astream({"topic": "bears"}):
    print(s, end="", flush=True)

One interesting fact about bears is that they have an exceptional sense of smell; it's estimated to be seven times stronger than that of dogs. Grizzly bears can detect food from over a mile away and can smell scents from up to 20 miles away under the right conditions. This incredible olfactory capability is crucial for their survival, helping them locate food sources, even in dense forest environments or across wide distances.

In [14]:
await chain.ainvoke({"topic": "bears"})

'An interesting fact about bears is that they have a highly developed sense of smell, which is believed to be seven times stronger than that of a bloodhound. This remarkable sense helps them locate food from great distances, find mates, and detect predators. In fact, some species of bears can smell food up to two miles away! This keen olfactory ability is crucial for their survival in the wild.'

In [15]:
await chain.abatch([{"topic": "bears"}, {"topic": "cats"}])

['One interesting fact about bears is that they have an excellent sense of smell, which is estimated to be cuburing more than 2,000 times better than that of humans. This remarkable olfactory ability allows them to detect food from miles away, making them proficient foragers. In fact, some species of bears can even smell food that is buried under several feet of snow! This heightened sense of smell plays a crucial role in their survival by helping them find food sources in their habitats.',
 "One interesting fact about cats is that they have a unique collarbone structure. Unlike humans and many other animals, a cat's collarbone (clavicle) is free-floating, which allows them to squeeze through narrow spaces and tight spots. This anatomical feature is part of what makes cats such agile and graceful creatures, as they can fit through openings as narrow as their heads!"]

### Retrieval Augmented Generation with LCEL

In [16]:
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough

embedding_function = OpenAIEmbeddings()

docs = [
    Document(
        page_content="the dog loves to eat pizza", metadata={"source": "animal.txt"}
    ),
    Document(
        page_content="the cat loves to eat lasagna", metadata={"source": "animal.txt"}
    ),
]


db = Chroma.from_documents(docs, embedding_function)
retriever = db.as_retriever()

In [17]:
retriever.invoke("What does the dog want to eat?")  # old method

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(id='40eefef5-380e-4cbc-ab48-8dbaa3840bc0', metadata={'source': 'animal.txt'}, page_content='the dog loves to eat pizza'),
 Document(id='aee55e87-bd42-4e66-9679-96583e42570a', metadata={'source': 'animal.txt'}, page_content='the cat loves to eat lasagna')]

In [18]:
retriever.invoke("What does the dog want to eat?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(id='40eefef5-380e-4cbc-ab48-8dbaa3840bc0', metadata={'source': 'animal.txt'}, page_content='the dog loves to eat pizza'),
 Document(id='aee55e87-bd42-4e66-9679-96583e42570a', metadata={'source': 'animal.txt'}, page_content='the cat loves to eat lasagna')]

In [19]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4o-mini")

In [20]:
from operator import itemgetter

retrieval_chain = (
    {
        "context": (lambda x: x["question"]) | retriever,
        # "question": lambda x: x["question"],
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | StrOutputParser()
)

In [21]:
retrieval_chain.invoke({"question": "What does the dog like to eat?"})

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'The dog likes to eat pizza.'

In [22]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4o-mini")

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [23]:
retrieval_chain.invoke("What does the dog like to eat?")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


'The dog loves to eat pizza.'